In [1]:
%cd ../src

/Users/shanekercheval/repos/sik-llms/src


# Clients

In [2]:
# For "registered" clients (via `@Client.register`), the client
# can be created with `create_client` by passing in the model name.
from sik_llms import create_client

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [3]:
# Or, the client can be directly instantiated
from sik_llms import OpenAI
client = OpenAI(
    model_name='gpt-4o-mini',
    temperature=0.1,
)
client

In [4]:
# Or, the client can be directly instantiated
from sik_llms import Anthropic
client = Anthropic(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
client

# Chat

In [5]:
from sik_llms import create_client, user_message, TextChunkEvent

client = create_client(
    model_name='gpt-4o-mini',
    temperature=0.1,
)

message = user_message("What is the capital of France?")
message

{'role': 'user', 'content': 'What is the capital of France?'}

### Run Synchronously

In [6]:
client(messages=[message])

ResponseSummary(input_tokens=14, output_tokens=7, input_cost=2.1e-06, output_cost=4.2e-06, duration_seconds=0.5737650394439697, response='The capital of France is Paris.')

### Stream Asynchronously

In [7]:
responses = []
async for response in client.run_async(messages=[message]):
    if isinstance(response, TextChunkEvent):
        print(response.content, end="")
        responses.append(response)

The capital of France is Paris.

In [8]:
print(response)

input_tokens=14 output_tokens=7 input_cost=2.1e-06 output_cost=4.2e-06 duration_seconds=0.6222331523895264 response='The capital of France is Paris.'


### Claude 

In [9]:
client = create_client(
    model_name='claude-3-7-sonnet-latest',
    temperature=0.1,
)
response = client(messages=[user_message("What is the capital of France?")])
response

ResponseSummary(input_tokens=14, output_tokens=49, input_cost=4.2000000000000004e-05, output_cost=0.000735, duration_seconds=0.9837002754211426, response="The capital of France is Paris. It's not only the capital city but also the largest city in France, known for its iconic landmarks like the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.")

# OpenAI Functions/Tools

In [ ]:
from sik_llms import (
    create_client, user_message,
    Tool, Parameter, RegisteredClients,
)

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            param_type='string',
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.OPENAI_TOOLS,
    model_name='gpt-4o-mini',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=60 output_tokens=17 input_cost=9e-06 output_cost=1.0199999999999999e-05 duration_seconds=0.6435151100158691 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='call_nDzxVHSjslnZcG2c9faDMEs5') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='call_nDzxVHSjslnZcG2c9faDMEs5'


---

# Claude Functions/Tools

In [ ]:
from sik_llms import (
    create_client, user_message,
    Tool, Parameter, RegisteredClients,
)

weather_tool = Tool(
    name='get_weather',
    description="Get the weather for a location.",
    parameters=[
        Parameter(
            name='location',
            param_type='string',
            required=True,
            description='The city and country for weather info.',
        ),
    ],
)

client = create_client(
    client_type=RegisteredClients.ANTHROPIC_TOOLS,
    model_name='claude-3-7-sonnet-latest',
    tools=[weather_tool],
)

message = user_message("What is the weather in Paris?")
response = await client.run_async(messages=[message])
# or `response = client(messages=[message])` for synchronous execution
print(response)
print('---')
print(response.tool_prediction)

input_tokens=394 output_tokens=40 input_cost=0.001182 output_cost=0.0006000000000000001 duration_seconds=1.3506109714508057 tool_prediction=ToolPrediction(name='get_weather', arguments={'location': 'Paris, France'}, call_id='toolu_01Eb6W2LtJnoxC1j4CpX32S3') message=None
---
name='get_weather' arguments={'location': 'Paris, France'} call_id='toolu_01Eb6W2LtJnoxC1j4CpX32S3'


---

# Structured Outputs via OpenAI

In [12]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='gpt-4o-mini',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = client(messages=messages)
print(response)
print('---')
print(response.response.parsed)

input_tokens=92 output_tokens=18 input_cost=1.38e-05 output_cost=1.08e-05 duration_seconds=0.5595331192016602 response=StructuredOutputResponse(parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']), refusal=None)
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Structured Outputs via Anthropic

In [13]:
from pydantic import BaseModel
from sik_llms import create_client, system_message, user_message


class CalendarEvent(BaseModel):  # noqa: D101
    name: str
    date: str
    participants: list[str]

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    response_format=CalendarEvent,
)
messages=[
    system_message("Extract the event information."),
    user_message("Alice and Bob are going to a science fair on Friday."),
]
response = client(messages=messages)
print(response)
print('---')
print(response.response.parsed)

input_tokens=458 output_tokens=78 input_cost=0.001374 output_cost=0.00117 duration_seconds=1.8412690162658691 response=StructuredOutputResponse(parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']), refusal=None)
---
name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

# Reasoning via OpenAI

In [14]:
from sik_llms import (
    create_client, user_message,
    TextChunkEvent, ResponseSummary, ReasoningEffort,
)

client = create_client(
    model_name='o3-mini',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None
async for response in client.run_async(messages=messages):
    if isinstance(response, TextChunkEvent):
        print(response.content, end="")
    elif isinstance(response, ResponseSummary):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")

To solve the expression 1 + 2 + (3 * 4) + (5 * 6), we first perform the multiplications inside the parentheses:

• 3 * 4 = 12  
• 5 * 6 = 30  

Now, substitute these results into the expression:

1 + 2 + 12 + 30

Next, add the numbers together:

1 + 2 = 3  
3 + 12 = 15  
15 + 30 = 45

So, the final result is 45.

In [15]:
summary

ResponseSummary(input_tokens=28, output_tokens=113, input_cost=3.08e-05, output_cost=0.0004972, duration_seconds=1.9797639846801758, response='To solve the expression 1 + 2 + (3 * 4) + (5 * 6), we first perform the multiplications inside the parentheses:\n\n• 3 * 4 = 12  \n• 5 * 6 = 30  \n\nNow, substitute these results into the expression:\n\n1 + 2 + 12 + 30\n\nNext, add the numbers together:\n\n1 + 2 = 3  \n3 + 12 = 15  \n15 + 30 = 45\n\nSo, the final result is 45.')

---

# Reasoning via Claude

In [16]:
from sik_llms import (
    create_client, user_message,
    TextChunkEvent, ThinkingChunkEvent,
    ResponseSummary, ReasoningEffort,
)

client = create_client(
    model_name='claude-3-7-sonnet-latest',
    reasoning_effort=ReasoningEffort.MEDIUM,
)
messages=[user_message("What is 1 + 2 + (3 * 4) + (5 * 6)?")]
summary = None

current_type = None
async for response in client.run_async(messages=messages):
    is_text_chunk = isinstance(response, TextChunkEvent)
    is_thinking_chunk = isinstance(response, ThinkingChunkEvent)
    is_summary = isinstance(response, ResponseSummary)

    if is_text_chunk or is_thinking_chunk:
        if type(response) is not current_type:
            print(f"\n\n[{'THINKING' if is_thinking_chunk else 'TEXT'}]")
            current_type = type(response)
        print(response.content, end="")
    elif isinstance(response, ResponseSummary):
        summary = response
    else:
        raise ValueError(f"Unexpected response type: {response}")



[THINKING]
Let me compute this step by step, following the order of operations (PEMDAS: Parentheses, Exponents, Multiplication/Division, Addition/Subtraction).

1 + 2 + (3 * 4) + (5 * 6)

First, I'll evaluate the expressions in parentheses:
(3 * 4) = 12
(5 * 6) = 30

Now I have:
1 + 2 + 12 + 30

Next, I'll perform the additions from left to right:
1 + 2 = 3
3 + 12 = 15
15 + 30 = 45

So the final result is 45.

[TEXT]
To solve this expression, I'll follow the order of operations (PEMDAS):

First, calculate the expressions in parentheses:
- (3 * 4) = 12
- (5 * 6) = 30

Then perform the additions:
1 + 2 + 12 + 30 = 45

Therefore, 1 + 2 + (3 * 4) + (5 * 6) = 45

---

# ReasoningAgent

In [ ]:
import json
from sik_llms.models_base import (
    Tool, Parameter, ThinkingEvent, ToolPredictionEvent,
    ToolResultEvent, TextChunkEvent, ErrorEvent, ResponseSummary,
)
from sik_llms.reasoning_agent import ReasoningAgent

####
# Define the tool functions
####
async def calculator(expression: str) -> str:
    """Execute calculator tool."""
    try:
        # Only allow simple arithmetic for safety
        allowed_chars = set('0123456789+-*/() .')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e!s}"


async def weather(location: str, units: str) -> str:
    """Mock weather tool - returns fake data."""
    # Return mock weather data
    weather_data = {
        'New York': '68',
        'San Francisco': '62',
        'Miami': '85',
        'Chicago': '55',
        'Los Angeles': '75',
    }
    for city in weather_data:  # noqa: PLC0206
        if city.lower() in location.lower():
            temp = weather_data[city]
            if units == 'C':
                # C = (°F - 32) x (5/9)

                temp = round((temp - 32) * 5 / 9)
            return {location: f"{temp}°{units}"}
    return None


####
# Define tool objects
####
calculator_tool = Tool(
    name='calculator',
    description="Perform mathematical calculations",
    parameters=[
        Parameter(
            name='expression',
            param_type='string',
            required=True,
            description="The mathematical expression to evaluate (e.g., '2 + 2', '5 * 10')",
        ),
    ],
    function=calculator,
)

weather_tool = Tool(
    name="get_weather",
    description="Get the current weather for a location",
    parameters=[
        Parameter(
            name="location",
            param_type="string",
            required=True,
            description="The name of the city (e.g., 'San Francisco', 'New York', 'London')",
        ),
        Parameter(
            name='units',
            param_type='enum',
            required=True,
            description="The units for temperature",
            enum=['F', 'C'],
        ),
    ],
    function=weather,
)


tool_executors = {
    "calculator": calculator,
    "get_weather": weather
}

# Create the reasoning agent
agent = ReasoningAgent(
    model_name="gpt-4o-mini",  # You can change this to other models
    tools=[calculator_tool, weather_tool],
    tool_executors=tool_executors,
    max_iterations=10,
    temperature=0,
)

question = "I'm planning a trip to New York and Miami. What's the weather like in both cities? Also, if I have a budget of $2400 for a 6-day trip, how much can I spend per day?"  # noqa: E501
# Run the agent and collect the results
messages = [{"role": "user", "content": question}]

print(f"[QUESTION]: {question}\n")

current_iteration = 0

async for event in agent.run_async(messages):
    if isinstance(event, ThinkingEvent):
        if hasattr(event, 'iteration') and event.iteration != current_iteration:
            current_iteration = event.iteration
            print(f"\n--- Iteration {current_iteration}\n")
        if event.content:
            print(f"\n[THINKING]:\n{event.content}")

    elif isinstance(event, ToolPredictionEvent):
        print("\n[TOOL PREDICTION]:")
        print(f"Tool: {event.name}`")
        print(f"Parameters: \n```json\n{json.dumps(event.arguments, indent=2)}\n```")

    elif isinstance(event, ToolResultEvent):
        print("\n[TOOL RESULT]:")
        print(f"Tool: {event.name}`")
        print(f"Result: {event.result}")

    elif isinstance(event, ErrorEvent):
        print("\n[ERROR]:")
        print(f"Error: {event.content}")

    elif isinstance(event, TextChunkEvent):
        # For streaming text generation
        if current_iteration  >= 0:  # Only print once for the header
            print("\n[FINAL RESPONSE]:")
            current_iteration = -1  # Set to an impossible iteration to avoid repeating

        print(event.content, end="")

    elif isinstance(event, ResponseSummary):
        # Print nothing here as we've already streamed the response
        pass

# Just for notebook display cleanliness
print("\n\n---\n\n")

ValueError: All tools must have a callable function

In [ ]:
print(f"Input Tokens: {event.input_tokens}")
print(f"Output Tokens: {event.output_tokens}")
print(f"Total Cost: {event.total_cost}")
print(f"Duration: {event.duration_seconds:.2f} seconds")